# Model Train

In [1]:
import pandas as pd
import numpy as np
import os
from gensim.models import Word2Vec

## 1. 데이터
- 명사 단위 데이터

In [3]:
data = pd.read_csv("data/NOUN_수정.txt").rename(columns={"Unnamed: 0":"ID"}).dropna() # NA 드랍

## 2. 모델
- gensim Word2Vec - skipgram & cbow

### 카테고리별 모델링

In [6]:
def modelTrain(cat, sg, path):
    if not os.path.exists(path):
        os.mkdir(path) # 모델 저장할 경로 생성
    df = data[data.Category == cat] # 카테고리 필터
    words = [str(x).split(" ") for x in df.Nouns.values] # 훈련용 데이터
    model = Word2Vec(words, window=3, min_count=3, size=100, sg=sg) # 모델 훈련
    model.init_sims(replace=True) # 불필요한 메모리 deload
    filename = cat.replace("/","") # 파일이름 지정
    model.save(f"{path}/{filename}.model") # 모델 저장

In [7]:
pd.Series(data.Category.unique()).apply(modelTrain, sg=1, path='./models/skipgram') # 카테고리별로 훈련, 저장

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
dtype: object

In [8]:
pd.Series(data.Category.unique()).apply(modelTrain, sg=0, path='./models/cbow') # 카테고리별로 훈련, 저장

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
dtype: object

### 전체 문서 모델링

In [4]:
words = [str(x).split(" ") for x in data.Nouns.values]

In [5]:
model = Word2Vec(words, window=3, min_count=3, size=100, sg=1) # skip-gram 방식으로 모델링
model.init_sims(replace=True) # 불필요한 메모리 deload
model.save("./models/skipgram/total.model")

In [6]:
model = Word2Vec(words, window=3, min_count=3, size=100, sg=0) # cbow 방식으로 모델링
model.init_sims(replace=True) # 불필요한 메모리 deload
model.save("./models/cbow/total.model")

## 3. 결과 정리

- '정의'와 가장 가까운 명사

In [18]:
def loadAndVal(path, topn=50):
    mostSimilar = Word2Vec.load(path).wv.most_similar("정의", topn=topn)
    
    result = pd.DataFrame({
        "model":path.split("/")[-2],
        "category":path.split("/")[-1].split(".")[0],
        "rank":np.arange(1,51),
        "keyword":[word for word, distance in mostSimilar],
        "distance":[distance for word, distance in mostSimilar],
        
        
    })
    
    return result

In [19]:
sgModels = [f"./models/skipgram/{filename}" for filename in os.listdir("./models/skipgram/")]
sgResult = pd.concat(list(pd.Series(sgModels).apply(loadAndVal)))

In [23]:
cbowModels = [f"./models/cbow/{filename}" for filename in os.listdir("./models/cbow/")]
cbowResult = pd.concat(list(pd.Series(cbowModels).apply(loadAndVal)))

In [26]:
pd.concat([sgResult, cbowResult]).to_csv("./정의단어/정의단어.csv", index=False)